In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
from tqdm.notebook import tqdm
import pandas as pd
import pymysql

In [200]:
driver = webdriver.Chrome(service=Service('driver/chromedriver'))

url = 'https://www.kurly.com/search?sword=' + '감자'

driver.get(url)

In [201]:
html_full = driver.page_source

In [204]:
html_src = BeautifulSoup(html_full)
table_list = html_src.select("div.css-i3aepa.e1c07x4811")

In [90]:
len(table_list)

12

In [209]:
html_src.select('script')

[<script type="application/ld+json">{"@context":"https://schema.org","@id":"https://www.kurly.com","@type":"Organization","name":"Kurly","url":"https://www.kurly.com/","logo":"https://res.kurly.com/images/marketkurly/logo/logo_x3.png","contactPoint":[{"@type":"ContactPoint","telephone":"+82-1644-1107","contactType":"customer support","areaServed":"KR","availableLanguage":["KO"]}],"sameAs":["https://www.instagram.com/marketkurly","https://www.youtube.com/channel/UCfpdjL5pl-1qKT7Xp4UQzQg","https://ko-kr.facebook.com/marketkurly","https://blog.naver.com/marketkurly","https://twitter.com/marketkurly","https://apps.apple.com/kr/app/id1080244833","https://play.google.com/store/apps/details?id=com.dbs.kurly.m2","https://www.youtube.com/channel/UCWaWvDRyhmSKOQWah7t7wuA"]}</script>,
 <script type="application/ld+json">{"@context":"http://schema.org","@type":"WebSite","url":"https://www.kurly.com/","potentialAction":{"@type":"SearchAction","target":"https://www.kurly.com/search?sword={searchword

In [167]:
for merch in table_list:
    name = str(merch.select('div.css-1qsrhd.e1c07x485')).split('\">')[1].split('</div>')[0]
    price = int(str(merch.select('span.sales-price.css-1qe668a.ei5rudb1')).split('\">')[1].split(' 원</')[0].replace(',',''))
    img_link = str(merch.select("div.e1c07x4810.css-1qpsnts.e3um3060")[0].select("img")[0]).split('src=\"')[1].split('\"/>')[0]
    
    print(name,',', price,',', img_link)

[KF365] 감자 1kg , 3800 , https://img-cf.kurly.com/cdn-cgi/image/width=676,format=auto/shop/data/goods/1637154387515l0.jpg
한끼 감자 300g , 1990 , https://img-cf.kurly.com/cdn-cgi/image/width=676,format=auto/shop/data/goods/1583288578136l0.jpg
친환경 감자 600g , 3290 , https://img-cf.kurly.com/cdn-cgi/image/width=676,format=auto/shop/data/goods/1653038027985l0.jpeg
서산 금왕 감자 900g , 5200 , https://img-cf.kurly.com/cdn-cgi/image/width=676,format=auto/shop/data/goods/1656391963198l0.jpg
친환경 깐 감자 , 4490 , https://img-cf.kurly.com/cdn-cgi/image/width=676,format=auto/shop/data/goods/1658380119521l0.jpg
감자 2kg , 7590 , https://img-cf.kurly.com/cdn-cgi/image/width=676,format=auto/shop/data/goods/1648208787398l0.jpeg
친환경 감자 2kg , 9900 , https://img-cf.kurly.com/cdn-cgi/image/width=676,format=auto/shop/data/goods/153543279519l0.jpg
친환경 깐 알감자 200g , 1500 , https://img-cf.kurly.com/cdn-cgi/image/width=676,format=auto/shop/data/goods/1645430132975l0.jpg
친환경 알감자 500g , 3500 , https://img-cf.kurly.com/cdn-cgi/im

In [267]:
# 한번에 마켓컬리 첫페이지 모든 정보 파싱하는 코드

driver = webdriver.Chrome(service=Service('driver/chromedriver'))
url = 'https://www.kurly.com/search?sword=' + 'asd'
driver.get(url)

time.sleep(10)

html_full = driver.page_source
html_src = BeautifulSoup(html_full)
table_list = html_src.select("div.css-i3aepa.e1c07x4811")

merch_list = pd.DataFrame()
name_list = []
price_list = []
img_link_list = []
link_list = []
o_o_o_list = []

for merch in table_list:
    name = str(merch.select('div.css-1qsrhd.e1c07x485')).split('\">')[1].split('</div>')[0]
    price = int(str(merch.select('span.sales-price.css-1qe668a.ei5rudb1')).split('\">')[1].split(' 원')[0].replace(',',''))
    img_link = str(merch.select("div.e1c07x4810.css-1qpsnts.e3um3060")[0].select("img")[0]).split('src=\"')[1].split('\"/>')[0]
    
    print(name,',', price,',', img_link)
    name_list.append(name)
    price_list.append(price)
    img_link_list.append(img_link)
    
for i in range(len(table_list)):
    xpath = '//*[@id="__next"]/div[3]/div/div[2]/div['+str(i+1)+']/div[2]/div[1]'
    driver.find_element(By.XPATH, xpath).click()
    
    time.sleep(5)
    
    html_full = driver.page_source
    html_src = BeautifulSoup(html_full)
    
    link = driver.current_url
    if '장바구니' in str(html_src.select('span.css-ymwvow.e4nu7ef1')) or '바로구매' in str(html_src.select('span.css-ymwvow.e4nu7ef1')):
        out_of_order = 0
    else:
        out_of_order = 1
    
    driver.back()
    time.sleep(10)
    
    print(link, out_of_order)
    link_list.append(link)
    o_o_o_list.append(out_of_order)
    
merch_list = pd.DataFrame({'name':name_list, 'cost':price_list, 'out_of_stock':o_o_o_list, 'link':link_list, 'img_link':img_link_list})
print(len(merch_list))
print(merch_list.isnull().values.any())

0
False


In [266]:
# 위 정보를 DB로 올리는 코드

try:
    conn,cur= None, None
    row='none'
    conn = pymysql.connect(host='127.0.0.1', user='ican1', password='1234', db='ssag', charset='utf8')
    cur = conn.cursor()

    for i in range(len(merch_list)):
        try:
            sql = "INSERT INTO merchandise_list (company_code, item_name, cost, out_of_stock, link, img_link) "
            sql+= "VALUES "
            sql+= '('+str(1)+',\"'+str(merch_list.loc[i][0])+'\",'+str(merch_list.loc[i][1])+','
            sql+= str(merch_list.loc[i][2])+',\"'+str(merch_list.loc[i][3])+'\",\"'+str(merch_list.loc[i][4])+'\");'
            print(sql)

            cur.execute(sql)
            conn.commit()
        except:
            None
            
finally:
    conn.close()

INSERT INTO merchandise_list (company_code, item_name, cost, out_of_stock, link, img_link) VALUES (1,"[Ozen] 훈제연어 슬라이스(허브딜) 180g (냉장)",16000,0,"https://www.kurly.com/goods/5070687","https://img-cf.kurly.com/cdn-cgi/image/width=676,format=auto/shop/data/goods/1618549909350l0.jpg");
INSERT INTO merchandise_list (company_code, item_name, cost, out_of_stock, link, img_link) VALUES (1,"[코리아보드게임즈] 할리갈리 컵스 딜럭스",19500,0,"https://www.kurly.com/goods/5149061","https://img-cf.kurly.com/cdn-cgi/image/width=676,format=auto/shop/data/goods/1657580930230l0.jpg");
INSERT INTO merchandise_list (company_code, item_name, cost, out_of_stock, link, img_link) VALUES (1,"[까사미아] 딜리스 플로어램프(단품)",540000,0,"https://www.kurly.com/goods/1000064698","https://3p-image.kurly.com/cdn-cgi/image/width=676,format=auto/product-image/3a84455f-5597-4d3a-9759-550b7f161a16/df0a111a-09a1-4e6d-aa89-466e9d560a43.jpg");
INSERT INTO merchandise_list (company_code, item_name, cost, out_of_stock, link, img_link) VALUES (1,"[Ozen] 훈제연

In [343]:
# SSG 판매상품 파싱

driver = webdriver.Chrome(service=Service('driver/chromedriver'))
url = 'https://www.ssg.com/search.ssg?target=all&query=' + '당근'
driver.get(url)

time.sleep(10)

html_full = driver.page_source
html_src = BeautifulSoup(html_full)
table_list = html_src.select("div.cunit_lst_v")[0].select('li.cunit_t232')[:12]

merch_list = pd.DataFrame()
name_list = []
price_list = []
img_link_list = []
link_list = []
o_o_o_list = []

for i in range(12):
    try:
        name = str(table_list[i].select('div.title > a > em')[0]).split('ko\">')[1].split('</em>')[0]
        price = int(str(table_list[i].select('div.opt_price > em')).split('price\">')[1].split("</em>")[0].replace(",",""))
        img_link = str(table_list[i].select("div.thmb > a > img")[0]).split('src="//')[1].split('\" width')[0]

        link = 'ssg.com' + str(table_list[i].select('div.thmb > a')[0]).split('href=\"')[1].split('&amp;siteNo')[0]
        if 'sold_out' in str(table_list[i]):
            out_of_order = 1
        else:
            out_of_order = 0
        print(name,',', price,',',out_of_order,',',link,',', img_link)

        name_list.append(name)
        price_list.append(price)
        o_o_o_list.append(out_of_order)
        link_list.append(link)
        img_link_list.append(img_link)
    except:
        continue
        
              
merch_list = pd.DataFrame({'name':name_list, 'cost':price_list, 'out_of_stock':o_o_o_list, 'link':link_list, 'img_link':img_link_list})
print(len(merch_list))
print(merch_list.isnull().values.any())

흙당근(봉) , 4980 , 0 , ssg.com/item/itemView.ssg?itemId=1000014896699 , sitem.ssgcdn.com/99/66/89/item/1000014896699_i1_232.jpg
친환경 당근 한뿌리 180g , 1980 , 0 , ssg.com/item/itemView.ssg?itemId=0000008104762 , sitem.ssgcdn.com/62/47/10/item/0000008104762_i1_232.jpg
친환경 당근 650g , 4980 , 0 , ssg.com/item/itemView.ssg?itemId=0000008326002 , sitem.ssgcdn.com/02/60/32/item/0000008326002_i1_232.jpg
흙 당근(1kg/봉) , 4980 , 0 , ssg.com/item/itemView.ssg?itemId=2097000204550 , sitem.ssgcdn.com/50/45/20/item/2097000204550_i1_232.jpg
[푸릇] 친환경 당근 500g , 3824 , 0 , ssg.com/item/itemView.ssg?itemId=2097000812861 , sitem.ssgcdn.com/61/28/81/item/2097000812861_i1_232.jpg
친환경 세척당근 500g , 4980 , 0 , ssg.com/item/itemView.ssg?itemId=0000007444235 , sitem.ssgcdn.com/35/42/44/item/0000007444235_i1_232.jpg
당근 1입(봉) 200g이상 , 2480 , 0 , ssg.com/item/itemView.ssg?itemId=2097000213767 , sitem.ssgcdn.com/67/37/21/item/2097000213767_i1_232.jpg
친환경 당근 500g/봉(2입봉내외) , 5980 , 0 , ssg.com/item/itemView.ssg?itemId=2097001166581

In [359]:
def ssgparsing(keyword):
    driver = webdriver.Chrome(service=Service('driver/chromedriver'))
    url = 'https://www.ssg.com/search.ssg?target=all&query=' + keyword
    driver.get(url)

    time.sleep(10)

    html_full = driver.page_source
    html_src = BeautifulSoup(html_full)
    table_list = html_src.select("div.cunit_lst_v")[0].select('li.cunit_t232')[:12]

    merch_list = pd.DataFrame()
    name_list = []
    price_list = []
    img_link_list = []
    link_list = []
    o_o_o_list = []

    for i in range(12):
        try:
            name = str(table_list[i].select('div.title > a > em')[0]).split('ko\">')[1].split('</em>')[0]
            price = int(str(table_list[i].select('div.opt_price > em')).split('price\">')[1].split("</em>")[0].replace(",",""))
            img_link = str(table_list[i].select("div.thmb > a > img")[0]).split('src="//')[1].split('\" width')[0]

            link = 'ssg.com' + str(table_list[i].select('div.thmb > a')[0]).split('href=\"')[1].split('&amp;siteNo')[0]
            if 'sold_out' in str(table_list[i]):
                out_of_order = 1
            else:
                out_of_order = 0
#             print(name,',', price,',',out_of_order,',',link,',', img_link)

            name_list.append(name)
            price_list.append(price)
            o_o_o_list.append(out_of_order)
            link_list.append(link)
            img_link_list.append(img_link)
        except:
            continue


    merch_list = pd.DataFrame({'name':name_list, 'cost':price_list, 'out_of_stock':o_o_o_list, 'link':link_list, 'img_link':img_link_list})
#     print(len(merch_list))
#     print(merch_list.isnull().values.any())
    company_code = 3
    return company_code, merch_list

In [365]:
def dbupload(company_code, merch_list):
    try:
        conn,cur= None, None
        row='none'
        conn = pymysql.connect(host='127.0.0.1', user='ican1', password='1234', db='ssag', charset='utf8')
        cur = conn.cursor()

        for i in range(len(merch_list)):
            try:
                sql = "INSERT INTO merchandise_list (company_code, item_name, cost, out_of_stock, link, img_link) "
                sql+= "VALUES "
                sql+= '('+str(company_code)+',\"'+str(merch_list.loc[i][0])+'\",'+str(merch_list.loc[i][1])+','
                sql+= str(merch_list.loc[i][2])+',\"'+str(merch_list.loc[i][3])+'\",\"'+str(merch_list.loc[i][4])+'\");'
#                 print(sql)

                cur.execute(sql)
                conn.commit()
            except:
                None

    finally:
        conn.close()
    
    return 

In [69]:
keyword = '감자'

driver = webdriver.Chrome(service=Service('driver/chromedriver'))
url = 'https://www.coupang.com/np/search?component=&q='+ keyword +'&channel=user'
driver.get(url)
time.sleep(10)

html_full = driver.page_source
html_src = BeautifulSoup(html_full)
table_list = html_src.select('div.search-content.search-content-with-feedback > ul')[0].select("li.search-product")

merch_list = pd.DataFrame()
name_list = []
price_list = []
img_link_list = []
link_list = []
o_o_o_list = []

for i in range(12):
    name = str(table_list[i].select('div.name')[0]).split('\">')[1].split('</div>')[0]
    price = int(str(table_list[i].select('strong.price-value')).split('\">')[1].split('</strong>')[0].replace(',',''))
    img_link = str(table_list[i].select('dl.search-product-wrap')[0].select('dt.image')[0]).split('src=\"//')[1].split('.jpg')[0] + '.jpg'
    link = 'https://www.coupang.com/vp/products/' + str(table_list[i]).split(' id=\"')[1].split('\" style')[0]

    if '일시품절' in str(table_list[i]):
        out_of_order = 1
    else:
        out_of_order = 0

    name_list.append(name)
    price_list.append(price)
    o_o_o_list.append(out_of_order)
    link_list.append(link)
    img_link_list.append(img_link)

merch_list = pd.DataFrame({'name':name_list, 'cost':price_list, 'out_of_stock':o_o_o_list, 'link':link_list, 'img_link':img_link_list})
company_code = 3

In [79]:
merch_list

,name,cost,out_of_stock,link,img_link
0,"미쁜스토어 포실포실한 2022년 햇 감자 3kg 5kg 10kg 20kg, 1개, ...",25900,0,"https://www.coupang.com/vp/products/188505703""...",thumbnail10.coupangcdn.com/thumbnails/remote/2...
1,"곰곰 국내산 감자(햇), 1kg",4590,0,https://www.coupang.com/vp/products/1074470750...,thumbnail8.coupangcdn.com/thumbnails/remote/23...
2,"국내산 감자(햇), 1kg, 1봉",4690,0,https://www.coupang.com/vp/products/4778058429...,thumbnail8.coupangcdn.com/thumbnails/remote/23...
3,"곰곰 국내산 감자(햇), 5kg",18900,0,https://www.coupang.com/vp/products/5233896155...,thumbnail6.coupangcdn.com/thumbnails/remote/23...
4,"경매직송 감자, 알뜰형, 80g-120g(대), 5kg",13500,0,https://www.coupang.com/vp/products/6726516775...,thumbnail7.coupangcdn.com/thumbnails/remote/23...
5,"히카마(얌빈) 멕시코감자 상급 10kg 다이어트채소 당뇨병 대장암 예방식품, 5kg",22500,0,https://www.coupang.com/vp/products/6819977423...,thumbnail6.coupangcdn.com/thumbnails/remote/23...
6,"한끼 감자(햇), 350g, 1봉",2800,0,https://www.coupang.com/vp/products/6202345577...,thumbnail10.coupangcdn.com/thumbnails/remote/2...
7,2022년 수확 국내산 감자 햇감자 수미감자 타박감자 강원도감자 3kg 5kg 10...,10800,0,https://www.coupang.com/vp/products/6272968059...,thumbnail7.coupangcdn.com/thumbnails/remote/23...
8,"네츄럴픽 2022년 하우스 햇 감자 수미감자 3kg 5kg 선별엄수, 1박스, 햇감...",13800,0,https://www.coupang.com/vp/products/1696806586...,thumbnail8.coupangcdn.com/thumbnails/remote/23...
9,"햇살조은 22년 햇 감자, 감자5kg(중), 1개",10710,0,"https://www.coupang.com/vp/products/1231177""><...",thumbnail10.coupangcdn.com/thumbnails/remote/2...


In [360]:
keyword='가지'
company_code, merch_list = ssgparsing(keyword)

In [364]:
dbupload(company_code, merch_list)